# Computaional Theory Problems

# Problem 1 — Binary Words and Bitwise Operations

This notebook implements the core 32-bit logic functions defined in the **Secure Hash Standard (FIPS PUB 180-4, § 4.1.2)**.
These functions are used in the SHA-256 algorithm to combine 32-bit words with rotations, shifts, and Boolean operations.

The goal is to:
1. Build helper functions that safely handle 32-bit operations in NumPy.
2. Implement `Parity`, `Ch`, `Maj`, `Σ₀`, `Σ₁`, `σ₀`, and `σ₁`.
3. Document and explain each step clearly.


### Step 1 – Secure 32-bit Helpers
We use NumPy’s `uint32` type to enforce 32-bit behavior.
The helper functions `_to_u32`, `_rotr`, and `_shr` guarantee logical rotation and right-shift operations.


In [32]:
import numpy as np
from typing import Union

Word = np.uint32

def _to_u32(x: Union[int, np.integer]) -> Word:
    """Force value to 32-bit unsigned word."""
    return Word(int(x) & 0xFFFFFFFF)

def _rotr(x: Word, n: int) -> Word:
    """Rotate-right 32-bit x by n."""
    x = _to_u32(x); n = int(n) % 32
    if n == 0:
        return x
    return _to_u32((x >> n) | (x << Word(32 - n)))

def _shr(x: Word, n: int) -> Word:
    """Logical right shift 32-bit x by n (zero-fill)."""
    x = _to_u32(x); n = int(n) % 32
    return _to_u32(x >> n)

### Step 2 – Boolean Functions
These operations mix three 32-bit words using logical bit rules:
- **Parity(x,y,z)** = x ⊕ y ⊕ z  
- **Ch(x,y,z)** = (x ∧ y) ⊕ (¬x ∧ z)  
- **Maj(x,y,z)** = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)


In [33]:
def Parity(x, y, z) -> Word:
    """Parity(x,y,z): XOR from 32-bit ( SHA-1 parity)."""
    return _to_u32(_to_u32(x) ^ _to_u32(y) ^ _to_u32(z))

def Ch(x, y, z) -> Word:
    """Ch (Eq. 4.2): (x & y) ^ (~x & z) — pick bit from y if bit x =1, otherwise z."""
    x, y, z = _to_u32(x), _to_u32(y), _to_u32(z)
    return _to_u32((x & y) ^ ((~x) & z))

def Maj(x, y, z) -> Word:
    """Maj (Eq. 4.3): (x & y) ^ (x & z) ^ (y & z) — majority base on each bit."""
    x, y, z = _to_u32(x), _to_u32(y), _to_u32(z)
    return _to_u32((x & y) ^ (x & z) ^ (y & z))


### Step 3 – Σ and σ Functions
These are rotation/shift-based transformations defined in FIPS 180-4:
- **Σ₀(x)** = ROTR² ⊕ ROTR¹³ ⊕ ROTR²²  
- **Σ₁(x)** = ROTR⁶ ⊕ ROTR¹¹ ⊕ ROTR²⁵  
- **σ₀(x)** = ROTR⁷ ⊕ ROTR¹⁸ ⊕ SHR³  
- **σ₁(x)** = ROTR¹⁷ ⊕ ROTR¹⁹ ⊕ SHR¹⁰


In [ ]:
def Sigma0(x) -> Word:
    """Σ₀: ROTR² ⊕ ROTR¹³ ⊕ ROTR²²."""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22))

def Sigma1(x) -> Word:
    """Σ₁: ROTR⁶ ⊕ ROTR¹¹ ⊕ ROTR²⁵."""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25))

def sigma0(x) -> Word:
    """σ₀: ROTR⁷ ⊕ ROTR¹⁸ ⊕ SHR³."""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 7) ^ _rotr(x, 18) ^ _shr(x, 3))

def sigma1(x) -> Word:
    """σ₁: ROTR¹⁷ ⊕ ROTR¹⁹ ⊕ SHR¹⁰."""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 17) ^ _rotr(x, 19) ^ _shr(x, 10))


### Step 4 – Demonstration with Fixed Values
We test the functions using example 32-bit words from the SHA-256 initial values.


In [43]:
# Predefined 32-bit demo inputs
x = np.uint32(0x6a09e667)  # constant from SHA-256 IV
y = np.uint32(0x12345678)
z = np.uint32(0xdeadbeef)

print("===== INPUT VALUES =====")
print(f"x = {hex(int(x))}")
print(f"y = {hex(int(y))}")
print(f"z = {hex(int(z))}")

print("\n===== LOGIC FUNCTIONS =====")
print(f"Parity(x, y, z) = {hex(int(Parity(x, y, z)))}")
print(f"Ch(x, y, z)     = {hex(int(Ch(x, y, z)))}")
print(f"Maj(x, y, z)    = {hex(int(Maj(x, y, z)))}")

print("\n===== SIGMA FUNCTIONS =====")
print(f"Sigma0(x) = {hex(int(Sigma0(x)))}")
print(f"Sigma1(x) = {hex(int(Sigma1(x)))}")
print(f"sigma0(x) = {hex(int(sigma0(x)))}")
print(f"sigma1(x) = {hex(int(sigma1(x)))}")


===== INPUT VALUES =====
x = 0x6a09e667
y = 0x12345678
z = 0xdeadbeef

===== LOGIC FUNCTIONS =====
Parity(x, y, z) = 0xa6900ef0
Ch(x, y, z)     = 0x96a45ee8
Maj(x, y, z)    = 0x5a2df66f

===== SIGMA FUNCTIONS =====
Sigma0(x) = 0xce20b47e
Sigma1(x) = 0x55b65510
sigma0(x) = 0xba0cf582
sigma1(x) = 0xcfe5da3c


### Step 5 – Reflection and Research Discussion
According to **FIPS PUB 180-4** (NIST, 2015), these functions form the
non-linear mixing stage of SHA-256.  
Each rotation and shift ensures diffusion and bit independence.

Sources:
- [NIST FIPS 180-4 (2015) — Secure Hash Standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)
- Numpy documentation on [Unsigned integer types](https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32)


## Problem 2: Fractional Parts of Cube Roots

In [37]:
1

1

## Problem 3: Padding

In [38]:
1

1

## Problem 4: Hashes

In [39]:
1

1

## Problem 5: Passwords

In [40]:
1

1